In [1]:
from random import randint

class ECPoint():
    
    def __init__(self, x, y, inf=0):
        self.x = x
        self.y = y
        self.inf = inf

    def __eq__(self, other):
        if (self.inf == 1) and (other.inf == 1):
            return True

        return (self.x == other.x) and (self.y == other.y)

    def __repr__(self):
        if self.inf == 1:
            return 'O'
        # print the compressed version
        return '({}, {})'.format(self.x, self.y & 1)

    def __hash__(self):
        return hash(str(self))


class EllipticCurve():

    def __init__(self, p, g, a, b):
        """ A curve is defined by
            p: The finite field Fp
            g: The base point (generator) for the group
            a, b: Curve parameters, Y^2 = X^3 + aX + b
        """
        self.p = p
        self.g = g
        self.a = a
        self.b = b

    def identity(self):
        """ Returns hte identity element. """
        return ECPoint(0, 0, 1)

    def is_valid(self, p):
        """ Checks whether a point P is valid. """
        return p.y**2  % self.p == (p.x**3 + self.a*p.x + self.b) % self.p

    def random_point(self):
        """ Generate a random point (not identity) on the curve. """
        m = randint(1, self.p)
        p = self.mul(self.g, m)
        while p == self.identity():
            m = randint(1, self.p)
            p = self.mul(self.g, m)

        return p

    def egcd(self, a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = self.egcd(b % a, a)
            return (g, x - (b // a) * y, y)

    def modinv(self, a, m):
        g, x, y = self.egcd(a, m)
        if g != 1:
            raise Exception('Modular inverse does not exist')
        else:
            return x % m

    def add(self, p1, p2):
        """ Adds two points P1 = (x1, y1) and P2 = (x2, y2) on the given curve. """
        # special case if one point is O (identity)
        if p1.inf == 1 and p2.inf == 1:
            return self.identity()
        if p1.inf == 1:
            return p2
        if p2.inf == 1:
            return p1

        if p1.x != p2.x:
            lam = ((p2.y - p1.y) * self.modinv((p2.x - p1.x) % self.p, self.p)) % self.p
        else:
            if (p1 == self.neg(p2)):
                return self.identity()
            if (p1.y == 0):
                return self.identity()
            
            # point doubling
            lam = ((3*p1.x**2 + self.a) * self.modinv(2 * p1.y, self.p)) % self.p


        x3 = (lam**2 - p1.x - p2.x) % self.p
        y3 = ((p1.x - x3) * lam - p1.y) % self.p
        return ECPoint(x3, y3)

    def neg(self, p):
        """ Calculate the additive inverse of a point P1, -P1. """
        return ECPoint(p.x, self.p - p.y)

    def sub(self, p1, p2):
        """ Subtract P2 from P1, i.e., P1 - P2 = P1 + (-P2). """
        return self.add(p1, self.neg(p2))

    def mul(self, p, m):
        """ Multiply a point P with a constant m, using double-and-add. """
        result = self.identity()
        addend = p

        while m:
            if m & 1:
                result = self.add(result, addend)

            # double the point
            addend = self.add(addend, addend)
            m >>= 1

        return result

In [2]:
from random import randint
from hashlib import sha256

# from ecc import EllipticCurve, ECPoint

def keygen():
    # use the secp256k1 curve
    p = int('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F', 16)
    a = 0
    b = 7
    g = ECPoint(int('79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798', 16), 
        int('483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8', 16))
    G = EllipticCurve(p, g, a, b)

    # generate private key
    x = randint(1, p)
    h = G.mul(g, x)

    return (x, G, g, p, h)


def encrypt(m, G, g, p, h):
    """
    Encryption of plaintext m.
    Parameters
    ----------
    m: The message, a point on the curve
    G: The curve
    g: The curve base point
    p: The order of the field
    h: Public part of the shared secret
    """

    # TODO: implement a padding scheme for m, like OAEP

    y = randint(1, p)
    c1 = G.mul(g, y)
    s = G.mul(h, y)         # h*y = g*xy
    # c2 = m xor H(h*y)
    hs = sha256(repr(s).encode('utf-8')).digest()
#     print (hs)
    c2 = bytearray([i ^ j for i,j in zip(m, hs)])
    return (c1, bytes(c2))


def decrypt(c, x, G):
    """
    Decryption of ciphertext c.
    
    Parameters
    ----------
    c: The ciphertext tuple, (c1, c2)
    x: The private key
    G: The curve
    """
        
    c1, c2 = c
    s = G.mul(c1, x)
    # m = c2 xor H(c1*x)
    hs = sha256(repr(s).encode('utf-8')).digest()
    m = bytearray([i ^ j for i,j in zip(c2, hs)])
    return bytes(m)


# if __name__ == '__main__':
def ecckey(m):
    x, G, g, p, h = keygen()
#     m = input('Enter message: ').encode('utf-8')
    m = m.encode('utf-8')

    c = encrypt(m, G, g, p, h)
    print('Encrypted: {}'.format(c))
    return c[1]
#     mp = decrypt(c, x, G)
#     print('Decrypted:\t{}'.format(mp.decode()))
#     assert m == mp

In [ ]:
from Crypto import Random
from Crypto.Cipher import AES
import os
import os.path
from os import listdir
from os.path import isfile, join
import time


class Encryptor:
    def __init__(self, key):
        self.key = key

    def pad(self, s):
        return s + b"\0" * (AES.block_size - len(s) % AES.block_size)

    def encrypt(self, message, key, key_size=256):
        print (message)
        message = self.pad(message)
        print (message)
        iv = Random.new().read(AES.block_size)
        print (iv)
        cipher = AES.new(key, AES.MODE_CBC, iv)
        print (iv + cipher.encrypt(message))
        return iv + cipher.encrypt(message)

    def encrypt_file(self, file_name):
        with open(file_name, 'rb') as fo:
            plaintext = fo.read()
        enc = self.encrypt(plaintext, self.key)
        with open(file_name + ".enc", 'wb') as fo:
            fo.write(enc)
        os.remove(file_name)

    def decrypt(self, ciphertext, key):
        iv = ciphertext[:AES.block_size]
        cipher = AES.new(key, AES.MODE_CBC, iv)
        plaintext = cipher.decrypt(ciphertext[AES.block_size:])
        return plaintext.rstrip(b"\0")

    def decrypt_file(self, file_name):
        with open(file_name, 'rb') as fo:
            ciphertext = fo.read()
        dec = self.decrypt(ciphertext, self.key)
        with open(file_name[:-4], 'wb') as fo:
            fo.write(dec)
        os.remove(file_name)

#     def getAllFiles(self):
#         dir_path = os.path.dirname(os.path.realpath(__file__))
#         dirs = []
#         for dirName, subdirList, fileList in os.walk(dir_path):
#             for fname in fileList:
#                 if (fname != 'script.py' and fname != 'data.txt.enc'):
#                     dirs.append(dirName + "\\" + fname)
#         return dirs

#     def encrypt_all_files(self):
#         dirs = self.getAllFiles()
#         for file_name in dirs:
#             self.encrypt_file(file_name)

#     def decrypt_all_files(self):
#         dirs = self.getAllFiles()
#         for file_name in dirs:
#             self.decrypt_file(file_name)


# key = b'[EX\xc8\xd5\xbfI{\xa2$\x05(\xd5\x18\xbf\xc0\x85)\x10nc\x94\x02)j\xdf\xcb\xc4\x94\x9d(\x9e'
key = '[EX\xc8\xd5\xbfI{\xa2$\x9e'
key = ecckey(key)
# key = str.encode(key)
enc = Encryptor(key)
clear = lambda: os.system('cls')

# if os.path.isfile('data.txt.enc'):
#     while True:
#         password = str(input("Enter password: "))
#         enc.decrypt_file("data.txt.enc")
#         p = ''
#         with open("data.txt", "r") as f:
#             p = f.readlines()
#         if p[0] == password:
#             enc.encrypt_file("data.txt")
#             break

# while True:
#     clear()
#     choice = int(input(
#         "1. Press '1' to encrypt file.\n2. Press '2' to decrypt file.\n3. Press '3' to Encrypt all files in the directory.\n4. Press '4' to decrypt all files in the directory.\n5. Press '5' to exit.\n"))
#     clear()
#     if choice == 1:
#         enc.encrypt_file(str(input("Enter name of file to encrypt: ")))
#     elif choice == 2:
#         enc.decrypt_file(str(input("Enter name of file to decrypt: ")))
#     elif choice == 3:
#         enc.encrypt_all_files()
#     elif choice == 4:
#         enc.decrypt_all_files()
#     elif choice == 5:
#         exit()
#     else:
#         print("Please select a valid option!")

# else:
#     while True:
#         clear()
#         password = str(input("Setting up stuff. Enter a password that will be used for decryption: "))
#         repassword = str(input("Confirm password: "))
#         if password == repassword:
#             break
#         else:
#             print("Passwords Mismatched!")
#     f = open("data.txt", "w+")
#     f.write(password)
#     f.close()
#     enc.encrypt_file("Data.txt")
#     print("Please restart the program to complete the setup")
#     time.sleep(15)

enc.encrypt_file(str(input("Enter name of file to encrypt: ")))
enc.decrypt_file(str(input("Enter name of file to decrypt: ")))

In [8]:
s = b'ab'
# s = str.encode(s)
type(s)

bytes